## **Importing Dependencies**

In [1]:
# Standrad dependencies
import cv2
import os
import random
import uuid
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, MaxPooling2D, Flatten, Layer
import tensorflow as tf

## **Creating Folder Structures**

In [3]:
pos_path = os.path.join('data', 'positive')
neg_path = os.path.join('data', 'negative')
anc_path = os.path.join('data', 'anchor')

In [ ]:
os.makedirs(pos_path)
os.makedirs(neg_path)
os.makedirs(anc_path)

## **Collecting Data**

### **Negative class**

In [ ]:
# Moving the lfw images into negative directory
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        source_path = os.path.join('lfw', directory, file)
        dest_path = os.path.join(neg_path, file)
        os.replace(source_path, dest_path)

### **Positive and Anchor class**

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, frame = cap.read()

    # Cropping frames
    frame = frame[130:130+250, 150:150+250, :]

    # Collecting anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        img_name = os.path.join(anc_path, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(img_name, frame)

    # Collecting positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        img_name = os.path.join(pos_path, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(img_name, frame)

    cv2.imshow('Image Collection', frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## **Loading & Preprocessing**

### **Loading Image Directories**

In [4]:
anchor = tf.data.Dataset.list_files(anc_path + '\\*.jpg').take(300)
positive = tf.data.Dataset.list_files(pos_path + '\\*.jpg').take(300)
negative = tf.data.Dataset.list_files(neg_path + '\\*.jpg').take(300)

### **Preprocessing**

In [5]:
def preprocess(file_path):
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)

    # Load in the image
    img = tf.io.decode_jpeg(byte_img) 
    
    # Resizing
    img = tf.image.resize(img, (105, 105))
    
    # Scaling
    img = img / 255.0
    
    return img

In [6]:
def preprocess_dataset(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)

### **Creating Labelled Dataset**

In [7]:
positives = tf.data.Dataset.zip(anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor))))
negatives = tf.data.Dataset.zip(anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor))))
data = positives.concatenate(negatives)

### **Train-Test Split**

In [8]:
# Building dataloader pipeline
data = data.map(preprocess_dataset)
data = data.cache()
data = data.shuffle(buffer_size = 1024) # Shuffling the dataset

In [9]:
# Training partition
train_data = data.take(round(len(data) * 0.7)) # 70% as training
train_data = train_data.batch(16) # Batch size
train_data = train_data.prefetch(8) # Prefetches the next 8 when processing the previous batch

In [19]:
# Testing partition
test_data = data.skip(round(len(data) * 0.7))
test_data = test_data.take(round(len(data) * 0.3))
test_data = test_data.batch(16) # Batch size
test_data = test_data.prefetch(8) # Prefetches the next 8 when processing the previous batch